# Includes

In [ ]:
import re
import nltk
import spacy
import torch
import pickle
import optuna
import scikitplot
import matplotlib
import unicodedata

import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt

nltk.download('punkt')              # Download to be able to use word_tokenize from nltk library

!pip install optuna                 # Download to be able to use Optuna as hyper-parameter
!python -m spacy download el        # Download to be able to lemmatize greek words using spacy
!pip install greek-stemmer-pos      # Download to be able to use stem on greek words

from tabulate import tabulate
from wordcloud import WordCloud
from collections import Counter
from greek_stemmer import stemmer

from gensim.models import Word2Vec

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from torch.utils.data import TensorDataset, DataLoader

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

# Datasets

In [ ]:
# ---- Reading the datasets ----

df_test = pd.read_csv('/kaggle/input/ys19-2023-assignment-3/test_set.csv')
df_train = pd.read_csv('/kaggle/input/ys19-2023-assignment-3/train_set.csv')
df_valid = pd.read_csv('/kaggle/input/ys19-2023-assignment-3/valid_set.csv')

# Data pre-processing

In [ ]:
# ---- NaN values ----

df_test.dropna(inplace = True)   # Good practice to
df_train.dropna(inplace = True)  # remove all empty   
df_valid.dropna(inplace = True)  # values of the datasets

In [ ]:
# ---- Stopwords ----

nltk.download('stopwords')
nlp = spacy.load('el_core_news_sm')

spacy_greek_stopwords = nlp.Defaults.stop_words       # Greek stopwords from spacy
nltk_greek_stopwords = set(stopwords.words('greek'))  # Greek stopwords from nltk

english_stopwords = set(stopwords.words('english'))
greek_stopwords = spacy_greek_stopwords.union(nltk_greek_stopwords) # Combination of those so there is a bigger list of stopwords

In [ ]:
# ---- Cleansing / Tokenizing ----

mapping_names = {
    'αλεξη' : 'αλεξης',
    'τσιπρα' : 'τσιπρας',
    'συριζας' : 'συριζα',
    'κουλη' : 'κυριακος',
    'κουλης' : 'κυριακος',
    'κυριακο' : 'κυριακος',
    'μητσοτακη' : 'μητσοτακης',
}

def replace_words(text):
    words = text.split()                                                # If the word is inside the dictionary (map for names)
    replaced_words = [mapping_names.get(word, word) for word in words]  # change it. Lemmatizer can't convert those names and if 
    return ' '.join(replaced_words)                                     # there's no usage of lemmatizer names must be equal

def split_connected(text):
    english_letters = r'[A-Za-z]+'
    greek_letters = r'[Α-Ωα-ωίϊΐόάέύϋΰήώ]+'
    
    greek = re.findall(greek_letters, text)     # Go and find the greek and the english 
    english = re.findall(english_letters, text) # words based on the patern above
    
    return ' '.join(english + greek)  # End combine them to have cleaner text. For example "Greekεκλογες" will be "Greek εκλογες"

def remove_accent(text):
    normalized = unicodedata.normalize('NFD', text)                              # Using unicodedata to decompose characters
    new_text = [char for char in normalized if not unicodedata.combining(char)]  # with tones or any accent and be able to keep
    return ''.join(new_text)

def stem_text(text):
    words = text.split()                                               # Split the text into words
    stemmed_words = [stemmer.stem_word(word,'VBG') for word in words]  # stem every word of the text 
    return ' '.join(stemmed_words)                                     # and join all text pieces

def lemmatize_text(text):
    tokens = nlp(text)                                     # Tokenize the text
    lemmatized_words = [token.lemma_ for token in tokens]  # then apply lemmatization
    return ' '.join(lemmatized_words)                      # and join all the tokens

def text_cleaning(column):
    column = column.str.lower()                                # All to lowercase first
    column = column.str.replace(r'[@#]', '', regex = True)     # Remove all mentions and hashtags
    column = column.str.replace(r'www\S+', '', regex = True)   # Remove anything that 
    column = column.str.replace(r'http\S+', '', regex = True)  # has to do with links 
    column = column.str.replace(r'[^\w\s]', '', regex = True)  # Remove all punctuation marks
       
    column = column.apply(split_connected)
    
    column = column.apply(lambda x: ' '.join(x.split())) # Replace consecutive whitespaces so the resulting description will contain a single space between words
    column = column.apply(lambda x: re.sub(r'(.)\1{2,}', r'\1\1', x)) # Replace 3 or more same consecutive characters by 2 using regular expressions (e.g balll will become ball)
    column = column.apply(lambda x: ' '.join([word for word in x.split() if word not in greek_stopwords]))   # Remove greek stopwords
    column = column.apply(lambda x: ' '.join([word for word in x.split() if word not in english_stopwords])) # Remove english stopwords

#     column = column.apply(stem_text)
#     column = column.str.lower()
#     column = column.apply(lemmatize_text)
    column = column.apply(remove_accent)
    column = column.apply(replace_words)
    
    return column

df_test['Text'] = text_cleaning(df_test['Text'])
df_train['Text'] = text_cleaning(df_train['Text'])
df_valid['Text'] = text_cleaning(df_valid['Text'])

df_test['Tokens'] = df_test['Text'].apply(word_tokenize)
df_train['Tokens'] = df_train['Text'].apply(word_tokenize)
df_valid['Tokens'] = df_valid['Text'].apply(word_tokenize)

df_train['Sentiment_encoded'] = LabelEncoder().fit_transform(df_train['Sentiment'])
df_valid['Sentiment_encoded'] = LabelEncoder().fit_transform(df_valid['Sentiment'])

# Data Analysis

In [ ]:
# ---- Pie Plots ----

plt.figure(figsize = (20, 8))

# What is the distribution of the sentiment
total_sentiment_distribution = df_train['Sentiment'].value_counts()

plt.subplot(1, 3, 1)
total_sentiment_distribution.plot(kind = 'pie', autopct = '%1.1f%%')
plt.title('Sentiments')

# Whats the total votes every party has
total_votes_party = df_train['Party'].value_counts()

plt.subplot(1, 3, 2)
total_votes_party.plot(kind = 'pie', autopct = lambda p: '{:.0f}'.format(p * total_votes_party.sum() / 100))
plt.title('Party Votes')

# How many positive sentiments each party has
positive_sentiment_party = df_train[df_train['Sentiment'] == 'POSITIVE']['Party'].value_counts()

plt.subplot(1, 3, 3)
positive_sentiment_party.plot(kind = 'pie', autopct = lambda p: '{:.0f}'.format(p * positive_sentiment_party.sum() / 100))
plt.title('Positive Sentiments by Party')

# Show all analysis #
plt.show()

In [ ]:
# ---- Bar Plots ----

plt.figure(figsize = (20, 8))

def find_top_words(column):
    count = Counter(word for words in column for word in words)      # Calculate word frequencies using Counter
    top_words = count.most_common(15)                                # and select the top 10 of them to show
    return pd.DataFrame(top_words, columns = ['Word', 'Frequency'])  # Create a DataFrame for plotting

# Top words used on Tweeter for the Greek Elections 
top_tweet_words = find_top_words(df_train['Tokens'])

plt.bar(top_tweet_words['Word'], top_tweet_words['Frequency'])
plt.xticks(rotation = 40)
plt.title('Top words Tweeted')
plt.xlabel('Word')
plt.ylabel('Frequency')

# Show all analysis #
plt.show()

In [ ]:
# ---- WordCloud for all dataset's text ----

plt.figure(figsize = (20, 8))

cloud = WordCloud(width = 800, height = 400, background_color = 'black').generate(' '.join(df_train['Text']))

plt.imshow(cloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('WordCloud')

In [ ]:
# ---- WordCloud for each Party ----

plt.figure(figsize = (20, 5))

parties = df_train['Party'].unique()

fig, axes = plt.subplots(nrows = (len(parties) // 3) + (len(parties) % 3 > 0), ncols = 3, figsize = (18, 6)) # Many word clouds so split for plotting

for i, party in enumerate(parties, start = 1):
    cloud = WordCloud(width = 800, height = 400, background_color = 'black').generate(' '.join(df_train[df_train['Party'] == party]['Text']))
    
    if len(parties) > 3:    # In a dataset im not
        row = (i - 1) // 3  # sure how many parties there 
        col = (i - 1) % 3   # are maybe it will come in handy
        ax = axes[row, col]
    else:
        ax = axes[i - 1]
    
    ax.imshow(cloud, interpolation = 'bilinear')
    ax.axis('off')
    ax.set_title(f'{party} Word Cloud')

plt.tight_layout()
plt.show()

In [ ]:
# ---- WordCloud for each Sentiment ----

plt.figure(figsize = (20, 5))

sentiments = df_train['Sentiment'].unique()

fig, axes = plt.subplots(nrows = (len(sentiments) // 3) + (len(sentiments) % 3 > 0), ncols = 3, figsize = (18, 6))

for i, sentiment in enumerate(sentiments, start = 1):
    cloud = WordCloud(width = 800, height = 400, background_color = 'black').generate(' '.join(df_train[df_train['Sentiment'] == sentiment]['Text']))
    
    if len(sentiments) > 3: # In a dataset im not
        row = (i - 1) // 3  # sure how many sentiments there
        col = (i - 1) % 3   # are maybe it will come in handy
        ax = axes[row, col]
    else:
        ax = axes[i - 1]
    
    ax.imshow(cloud, interpolation = 'bilinear')
    ax.axis('off')
    ax.set_title(f'{sentiment} Word Cloud')

plt.tight_layout()
plt.show()

In [ ]:
# ---- Word Cloud based on parties ----

plt.figure(figsize = (20, 8))

# WordCloud for the top 3 Political Parties
column = df_train[df_train['Party'].isin(['SYRIZA', 'ND', 'PASOK'])]['Text']
cloud = WordCloud(width = 800, height = 400, background_color = 'black').generate(' '.join(column))

plt.imshow(cloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Top 3 Parties Word Cloud')

# Vectorization, Word Embeddings & Data partioning

In [ ]:
# ---- Embeddings, Train / Valid (test) ----

def word_embeddings(column, model):
    embedding_vector = np.zeros((len(column), model.vector_size)) # Initialization of embeddings with zeros
    word_count = [len(sublist) for sublist in column]             # Count the words of every text

    for i, sentence in enumerate(column):
        embeddings = [model.wv[word] for word in sentence if word in model.wv] # .wv is vector that describes the word in numbers
        if word_count[i] > 0:
            embedding_vector[i] = sum(embeddings) / word_count[i]
            
    return embedding_vector

vector = Word2Vec(df_train['Tokens'], vector_size = 200, seed = 42, epochs = 20)
vector.train(df_train['Tokens'], total_examples = len(df_train['Tokens']), epochs = vector.epochs)

X_train = word_embeddings(df_train['Tokens'], vector)
Y_train = df_train['Sentiment_encoded']
        
X_valid = word_embeddings(df_valid['Tokens'], vector)
Y_valid = df_valid['Sentiment_encoded']

# Tensors

In [ ]:
# ---- Saving to tensors ----

x_train = torch.tensor(X_train, dtype = torch.float).view(-1, 1, vector.vector_size) 
y_train = torch.tensor(Y_train.values, dtype = torch.long)
x_valid = torch.tensor(X_valid, dtype = torch.float).view(-1, 1, vector.vector_size)
y_valid = torch.tensor(Y_valid.values, dtype = torch.long)

# Neural Network

In [ ]:
# A class for our bidirectional RNN 
class RNN(nn.Module):
    def __init__(self, cell_type, input_size, hidden_size, num_layers, dropout, output_size):
        super(RNN, self).__init__()
        
        cells = {
          'LSTM' : nn.LSTM,
          'GRU'  : nn.GRU
        }
        
        torch.manual_seed(42)
        self.activation = nn.Sigmoid()
        self.linear = nn.Linear(hidden_size * 2, output_size)   # Doubling the hidden size because rnn is bidirectional 
        self.rnn = cells[cell_type](input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout,  
                                    bidirectional = True, batch_first = True)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        out = rnn_out[:, -1, :]
        out = self.activation(self.linear(out))
        return out

In [ ]:
# ---- Setting up criterion, optimizer, data ----

model = RNN('GRU', x_train.shape[2], 64, 2, 0.2, 3)

criterion = nn.CrossEntropyLoss()                           # Initialize criterion
optimizer = torch.optim.Adam(model.parameters(), lr = 5e-5) # Initialize optimizer

dataset = TensorDataset(x_train, y_train)                          # Class to represent the data as list of tensors
dataloader = DataLoader(dataset, batch_size = 64, shuffle = True)  # Initialize dataloader

# Neural Network Training

In [ ]:
for epoch in range(10):
    losses = []
    model.train()
    for x, y in dataloader:
        optimizer.zero_grad()       # Delete previously stored gradients
        
        logits = model(x)           # Perform forward propagation
        loss = criterion(logits, y) # Calculating the loss using criterion function
        losses.append(loss.item())
                       
        loss.backward()                                    # Perform back propagation starting from the loss calculated in this epoch
        nn.utils.clip_grad_norm_(model.parameters(), 1e-3) # Gradient clipping to prevent exploding gradients
        optimizer.step()                                   # Update model's weights based on the gradients calculated during backprop
        
    print(f'Epoch {epoch}: Loss = {sum(losses) / len(dataloader):.5f}')

# Validation

In [ ]:
model.eval()
with torch.no_grad():
    logits_valid = model(x_valid)
    probabilities = nn.functional.softmax(logits_valid, dim = 1)
    predictions = torch.argmax(probabilities, dim = 1)

accuracy = accuracy_score(y_valid.numpy(), predictions.numpy())
f1 = f1_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)                 
recall = recall_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)        
precision = precision_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)
                          
table = [['Method', 'Accuracy', 'F1-Score', 'Recall', 'Precision'],
         ['Predict', accuracy, f1, recall, precision]]

print(tabulate(table, headers = 'firstrow', tablefmt = 'fancy_grid'))

# Hyper-parameters

In [ ]:
def objective(trial):
    num_layers = trial.suggest_int('num_layers', 2, 3) 
    dropout = trial.suggest_float('dropout', 0.0, 0.5)  
    hidden_size =  trial.suggest_categorical('hidden_size', [64, 128])
    cell_type = trial.suggest_categorical('cell_type', ['LSTM', 'GRU'])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log = True)
    
    model = RNN(cell_type, x_train.shape[2], hidden_size, num_layers, dropout, 3)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    dataset = TensorDataset(x_train, y_train)
    dataloader = DataLoader(dataset, batch_size = 64, shuffle = True) 
    
    for epoch in range(10):
        model.train()
        for x, y in dataloader:
            optimizer.zero_grad()       # Delete previously stored gradients
            
            logits = model(x)           # Perform forward propagation
            loss = criterion(logits, y) # Calculating the loss using criterion function
            
            loss.backward()                                    # Perform back propagation starting from the loss calculated in this epoch
            nn.utils.clip_grad_norm_(model.parameters(), 1e-3) # Gradient clipping to prevent exploding gradients
            optimizer.step()                                   # Update model's weights based on the gradients calculated during backprop
                    
    model.eval()
    with torch.no_grad():
        logits_valid = model(x_valid)
        probabilities = nn.functional.softmax(logits_valid, dim = 1)
        predictions = torch.argmax(probabilities, dim = 1)

    accuracy = accuracy_score(y_valid.numpy(), predictions.numpy())
        
    return accuracy

study = optuna.create_study(direction = 'maximize', study_name = 'Christos study')
study.optimize(objective, n_trials = 100)
best_parameters = study.best_params

best_dropout = best_parameters['dropout']
best_lr = best_parameters['learning_rate']
best_cell_type = best_parameters['cell_type']
best_num_layers = best_parameters['num_layers']
best_hidden_size = best_parameters['hidden_size']

print('Best hyperparameters: ', best_parameters)

# Neural Network using Hyper-Parameters

In [ ]:
# ---- Setting up criterion, optimizer, data ----

model = RNN(best_cell_type, x_train.shape[2], best_hidden_size, best_num_layers, best_dropout, 3)

criterion = nn.CrossEntropyLoss()                              # Initialize criterion
optimizer = torch.optim.Adam(model.parameters(), lr = best_lr) # Initialize optimizer

dataset = TensorDataset(x_train, y_train)                           # Class to represent the data as list of tensors
dataloader = DataLoader(dataset, batch_size = 64, shuffle = True)   # Initialize dataloader

# Training using Hyper-Parameters

In [ ]:
for epoch in range(10):
    losses = []
    model.train()
    for x, y in dataloader:
        optimizer.zero_grad()       # Delete previously stored gradients
        
        logits = model(x)           # Perform forward propagation
        loss = criterion(logits, y) # Calculating the loss using criterion function
        losses.append(loss.item())
        
        loss.backward()                                    # Perform back propagation starting from the loss calculated in this epoch
        nn.utils.clip_grad_norm_(model.parameters(), 1e-3) # Gradient clipping to prevent exploding gradients
        optimizer.step()                                   # Update model's weights based on the gradients calculated during backprop
        
    print(f'Epoch {epoch}: Loss = {sum(losses) / len(dataloader):.5f}')

# Validation with Hyper-Parameters

In [ ]:
model.eval()
with torch.no_grad():
    logits_valid = model(x_valid)
    probabilities = nn.functional.softmax(logits_valid, dim = 1)
    predictions = torch.argmax(probabilities, dim = 1)

accuracy = accuracy_score(y_valid.numpy(), predictions.numpy())
f1 = f1_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)                 
recall = recall_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)        
precision = precision_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)
                          
table = [['Method', 'Accuracy', 'F1-Score', 'Recall', 'Precision'],
         ['Predict', accuracy, f1, recall, precision]]

print(tabulate(table, headers = 'firstrow', tablefmt = 'fancy_grid'))

# Learning Curve, ROC, Confusion Matrix

In [ ]:
score_train = []
score_valid = []
losses_train = []
losses_valid = []
    
for i in range(2):
    model = RNN('GRU', x_train.shape[2], 64, 2, 0.2, 3)
    # model = RNN(best_cell_type, x_train.shape[2], best_hidden_size, best_num_layers, best_dropout, 3)

    criterion = nn.CrossEntropyLoss()                               # Initialize criterion
    optimizer = torch.optim.Adam(model.parameters(), lr = 5e-5)     # Initialize optimizer

    dataset_train = TensorDataset(x_train, y_train)                               # Class to represent the data as list of tensors
    dataloader_train = DataLoader(dataset_train, batch_size = 64, shuffle = True) # Initialize dataloader

    dataset_valid = TensorDataset(x_valid, y_valid)                               # Class to represent the data as list of tensors
    dataloader_valid = DataLoader(dataset_valid, batch_size = 64, shuffle = True) # Initialize dataloader

    if i == 0:
        for epoch in range(10):
            model.train()
            for x, y in dataloader_train:
                optimizer.zero_grad()       # Delete previously stored gradients
                
                logits = model(x)           # Perform forward propagation
                loss = criterion(logits, y) # Calculating the loss using criterion function

                loss.backward()                                    # Perform back propagation starting from the loss calculated in this epoch
                nn.utils.clip_grad_norm_(model.parameters(), 1e-3) # Gradient clipping to prevent exploding gradients
                optimizer.step()                                   # Update model's weights based on the gradients calculated during backprop

            model.eval()
            with torch.no_grad():
                logits_train = model(x_train)
                probabilities = nn.functional.softmax(logits_train, dim = 1)
                predictions = torch.argmax(probabilities, dim = 1)

            f1_train = f1_score(y_train.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)
            score_train.append(f1_train)

            model.eval()
            with torch.no_grad():
                logits_valid = model(x_valid)
                probabilities = nn.functional.softmax(logits_valid, dim = 1)
                predictions = torch.argmax(probabilities, dim = 1)

            f1_valid = f1_score(y_valid.numpy(), predictions.numpy(), average = 'macro', zero_division = 1)
            score_valid.append(f1_valid)
        
    else:
        for epoch in range(10):
            losses = []
            model.train()
            for x, y in dataloader_train:
                optimizer.zero_grad()       # Delete previously stored gradients
                
                logits = model(x)           # Perform forward propagation
                loss = criterion(logits, y) # Calculating the loss using criterion function
                losses.append(loss.item())

                loss.backward()                                    # Perform back propagation starting from the loss calculated in this epoch
                nn.utils.clip_grad_norm_(model.parameters(), 1e-3) # Gradient clipping to prevent exploding gradients
                optimizer.step()                                   # Update model's weights based on the gradients calculated during backprop

            loss = sum(losses) / len(dataloader_train)
            losses_train.append(loss)

            losses = []
            model.eval()
            with torch.no_grad():
                for x, y in dataloader_valid:
                    logits = model(x)           # Perform forward propagation
                    loss = criterion(logits, y) # Calculating the loss using criterion function
                    losses.append(loss.item())

                loss = sum(losses) / len(dataloader_valid)
                losses_valid.append(loss)

plt.figure(figsize = (20, 8))

plt.subplot(1, 2, 1)
plt.plot(score_train, label = 'Training')
plt.plot(score_valid, label = 'Validation')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(losses_train, label = 'Training')
plt.plot(losses_valid, label = 'Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# ---- ROC Curve / AUC Score ----

auc = np.round(roc_auc_score(y_valid.numpy(), probabilities.detach().numpy(), multi_class = 'ovo'), 3)
print('Total AUC: ', auc)

scikitplot.metrics.plot_roc(y_valid.numpy(), probabilities.detach().numpy())
plt.show()

In [ ]:
# ---- Confusion Matrix ----

confusion_martix = metrics.confusion_matrix(y_valid.numpy(), predictions.numpy())

sns.heatmap(confusion_martix, annot = True, fmt = 'd', cmap = 'coolwarm')
plt.ylabel('Prediction', fontsize = 13)
plt.xlabel('Actual', fontsize = 13)
plt.title('Confusion Matrix', fontsize = 17)
plt.show()

# Final Predictions

In [ ]:
def replace_sentiment(column):
    map_sentiments = {
        0: 'NEGATIVE',
        1: 'NEUTRAL',
        2: 'POSITIVE',
    }
    return list(map(map_sentiments.get, column))

X_test = word_embeddings(df_test['Tokens'], vector)
x_test = torch.tensor(X_test, dtype = torch.float).view(-1, 1, vector.vector_size)

model.eval()
with torch.no_grad():
    logits_test = model(x_test)
    probabilities = nn.functional.softmax(logits_test, dim = 1)
    predictions = torch.argmax(probabilities, dim = 1)

df_test['Sentiment'] = predictions
df_test['Sentiment'] = replace_sentiment(df_test['Sentiment'])

In [ ]:
df_predictions = df_test[['New_ID', 'Sentiment']]                                            # The dataset to output needs only
df_predictions = df_predictions.rename(columns = {'New_ID': 'Id', 'Sentiment': 'Predicted'}) # the 2 columns of sentiments and ids
df_predictions.to_csv('submission.csv', index = False)